<a href="https://colab.research.google.com/github/Mova-2020/Phonemic-transcription-of-Ukrainian-texts/blob/main/Term_comparison_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')
# Load a pre-trained sentence transformer model

Helper functions for each criterion

In [ ]:
# 1. Exactness (cosine similarity between term and definition embeddings)
def exactness(term_embedding, definition_embedding):
    return cosine_similarity([term_embedding], [definition_embedding])[0][0]

In [ ]:
# 2. Essentiality (ratio of largest and second-largest entailment degrees from definition)
def essentiality(entailments):
    largest = max(entailments)
    second_largest = sorted(entailments, reverse=True)[1]
    return largest / second_largest if second_largest != 0 else float('inf')

In [ ]:
# 3. Plainness (ratio of subwords in term and definition)
def plainness(term, definition):
    term_subwords = set(term.split())
    definition_subwords = set(definition.split())
    common_subwords = term_subwords.intersection(definition_subwords)
    return len(common_subwords) / len(term_subwords) if term_subwords else 0

In [ ]:
# 4. Derivativity (Checking for specific endings and addable subwords)
def derivativity(term):
    if term.endswith(("nnja", "tja", "ння", "тя")):
        return 0  # Bad derivation if ending in "nnja" or "ttja"
    else:
        return 1  # Can add subwords (placeholder, needs more logic)

In [ ]:
# 5. Good sound phonotactic rule compliance)
def good_sound(term):
    bad_clusters = ["ngh", "shr", "zhr", "нг", "шр", "жр"]
    if any(cluster in term for cluster in bad_clusters):
        return 0
    return 1  # No bad clusters

In [ ]:
# 6. Systemic feature (Existence of meronyms or hypernyms in the dictionary)
def systemic_feature(term, dictionary_entries):
    related_entries = [entry for entry in dictionary_entries if term in entry]
    return len(related_entries)

In [ ]:
# 7. Organicity (Inverse of maximum subword length)
def organicity(term):
    subwords = term.split()
    max_length = max(len(subword) for subword in subwords)
    return 1 / max_length if max_length != 0 else 0

In [ ]:
# 8. Compatibility (Valence of the term or closest analogs)
def compatibility(term, term_valences):
    return term_valences.get(term, 0)  # Placeholder for valence calculation

In [ ]:
# 9. Unambiguity (Inverse of number of definitions for the term)
def unambiguity(definitions_count):
    return 1 / definitions_count if definitions_count != 0 else 0

In [ ]:
# 10. Nominativity (Formula-based calculation)
def nominativity(conjunctions, verb_endings):
    nconj = len(conjunctions)
    nend = len([ending for ending in verb_endings if ending in ["ty", "tysja", "tysj", "ти", "тися", "тись"]])
    return 1 / (1 + nconj + nend)

In [ ]:
# 11. Brevity (Inverse number of symbols in the term)
def brevity(term):
    return 1 / len(term) if len(term) != 0 else 0

Data input

In [ ]:
def get_terms_from_user():
    # Просить користувача ввести терміни та їх означення для порівняння
    while True:
        try:
            num_terms_input = input("Введіть кількість термінів для порівняння (за замовчуванням 2): ")
            num_terms = int(num_terms_input) if num_terms_input else 2  # Use 2 as default if input is empty
            break
        except ValueError:
            print("Неправильний ввід. Будь ласка, введіть число.")

    terms_and_definitions = {}  # Using a dictionary to store terms and definitions
    for i in range(num_terms):
        term = input(f"Введіть термін {i + 1}: ")
        definition = input(f"Введіть означення для терміна '{term}': ")
        terms_and_definitions[term] = definition  # Store term and definition in the dictionary
    return terms_and_definitions  # Return the dictionary


Scores

In [ ]:
def calculate_scores_for_terms(terms_and_definitions):
    # Обчислює бали для кожного терміна
    # Replace with your actual dictionary of terms.
    dictionary = {"термін1": "означення1", "термін2": "означення2"}

    # Replace with your actual valence data for terms.
    term_valences = {"термін1": 2, "термін2": 1}

    scores = {}
    for term, definition in terms_and_definitions.items():
        # Generate embeddings for term and definition
        term_embedding = model.encode(term)
        definition_embedding = model.encode(definition)

        # Calculate Exactness using cosine similarity
        exactness_score = cosine_similarity([term_embedding], [definition_embedding])[0][0]
        print(f"Cosine similarity for '{term}': {exactness_score}") # Print for debugging

        # Essentiality Calculation (placeholder - requires entailment logic)
        # Replace this with your actual entailment calculation
        entailments = [0.8, 0.6, 0.4]  # Example entailment degrees
        essentiality_score = essentiality(entailments)


        # Plainness calculation (using sets for intersection)
        term_subwords = set(term.lower().split())
        definition_subwords = set(definition.lower().split())
        common_subwords = term_subwords.intersection(definition_subwords)
        plainness_score = len(common_subwords) / len(term_subwords) if term_subwords else 0
        print(f"Common subwords for '{term}': {common_subwords}")  # Print for debugging

        # Initialize scores for the current term
        scores_term = {
            "Exactness": exactness_score,
            "Essentiality": essentiality_score,
            "Plainness": plainness_score,
            "Derivativity": derivativity(term),
            "Good Sound": good_sound(term),
            # Use the populated dictionary
            "Systemic Feature": systemic_feature(term, list(dictionary.keys())),
            "Organicity": organicity(term),
            # Use the populated term_valences
            "Compatibility": compatibility(term, term_valences),
            "Unambiguity": 1,  # Replace with actual calculation based on definition count
            "Nominativity": nominativity([], []),  # No conjunctions or verb endings in the term
            "Brevity": brevity(term),
        }
        scores[term] = scores_term
        #Assign scores_term to scores dictionary with term as key
    return scores


Comparison

In [ ]:
terms_and_definitions = get_terms_from_user()

Введіть кількість термінів для порівняння (за замовчуванням 2): 
Введіть термін 1: вага статистична
Введіть означення для терміна 'вага статистична': Число різних квантових станів з даною енергією. У випадку неперервного розподілу енергії – кількість станів у даному інтервалі значень енергії.
Введіть термін 2: кратність стану
Введіть означення для терміна 'кратність стану': Число різних квантових станів з даною енергією. У випадку неперервного розподілу енергії – кількість станів у даному інтервалі значень енергії.


In [ ]:
scores = calculate_scores_for_terms(terms_and_definitions)
#Call the calculate_scores_for_terms function to get the scores

Cosine similarity for 'вага статистична': 0.5460278987884521
Common subwords for 'вага статистична': set()
Cosine similarity for 'кратність стану': 0.6928146481513977
Common subwords for 'кратність стану': set()


In [ ]:
for term, term_scores in scores.items():
    print(f"Бали для терміна '{term}': {term_scores}")
    overall_score = sum(term_scores.values()) / len(term_scores)
    print(f"Загальний бал для терміна '{term}': {overall_score}")

Бали для терміна 'вага статистична': {'Exactness': 0.5460279, 'Essentiality': 1.3333333333333335, 'Plainness': 0.0, 'Derivativity': 1, 'Good Sound': 1, 'Systemic Feature': 0, 'Organicity': 0.09090909090909091, 'Compatibility': 0, 'Unambiguity': 1, 'Nominativity': 1.0, 'Brevity': 0.0625}
Загальний бал для терміна 'вага статистична': 0.5484336657300797
Бали для терміна 'кратність стану': {'Exactness': 0.69281465, 'Essentiality': 1.3333333333333335, 'Plainness': 0.0, 'Derivativity': 1, 'Good Sound': 1, 'Systemic Feature': 0, 'Organicity': 0.1111111111111111, 'Compatibility': 0, 'Unambiguity': 1, 'Nominativity': 1.0, 'Brevity': 0.06666666666666667}
Загальний бал для терміна 'кратність стану': 0.5639932508420462
